In [1]:
!uv pip install -qU sentence-transformers
!uv pip install -q transformers
!uv pip install -q datasets 
!uv pip install -q ipywidgets
!uv pip install -q pandas 

### Research

In the provided notebook, research will be conducted on the performance and methods for fine-tuning a local, open-source embedding model to achieve better performance in the Ukrainian language.

### Multilingual Distillation

We will start by researching the approach proposed in the following [research paper](https://arxiv.org/pdf/2004.09813). 

#### Teacher and student models

In [2]:
from sentence_transformers import SentenceTransformer

teacher_model_id = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
teacher_model = SentenceTransformer(teacher_model_id)
teacher_model.to('cuda')
teacher_model 

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [6]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import Transformer, Pooling 

student_model_id = 'FacebookAI/xlm-roberta-base'
transformer_module = Transformer(student_model_id)
pooling_module = Pooling(
    word_embedding_dimension=transformer_module.get_word_embedding_dimension(),
    pooling_mode_cls_token=True,
    pooling_mode_mean_tokens=False
)
student_model = SentenceTransformer(modules=[transformer_module, pooling_module])
student_model.to('cuda')
student_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [9]:
# assert that transformer config matched for student and teacher model
assert student_model[0].get_config_dict() == teacher_model[0].get_config_dict()

In [10]:
# assert that pooling config matches for student and teacher model
assert student_model[1].get_config_dict() == teacher_model[1].get_config_dict()

#### Dataset

In [26]:
from datasets import load_dataset

dataset = load_dataset('sentence-transformers/parallel-sentences-talks', 'en-uk')
dataset 

DatasetDict({
    dev: Dataset({
        features: ['english', 'non_english'],
        num_rows: 993
    })
    train: Dataset({
        features: ['english', 'non_english'],
        num_rows: 201883
    })
})

In [27]:
train_dataset = dataset['train']
train_dataset.set_format(type='pandas')
train_df = train_dataset[:]

In [28]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201883 entries, 0 to 201882
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   english      201883 non-null  object
 1   non_english  201883 non-null  object
dtypes: object(2)
memory usage: 3.1+ MB


In [29]:
train_df.head()

,english,non_english
0,I want you to know that I believe kids will ea...,"Я хочу, щоб ви знали, що я впевнений що діти б..."
1,I want you to know that there are farmers' mar...,"Я хочу, щоб ви знали, що є фермерські ринки як..."
2,"I want you to know that me, my brother and sis...","Я хочу, щоб ви знали, що я, мої брат та сестра..."
3,I try to share this everywhere I go.,"Я намагаюся поділитися цим всюди, куди б я не ..."
4,"Not too long ago, my uncle said that he offere...","Не так давно, мій дядько сказав, що він запроп..."


In [30]:
eval_dataset = dataset['dev']
eval_dataset.set_format(type='pandas')
eval_df = eval_dataset[:]

In [31]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   english      993 non-null    object
 1   non_english  993 non-null    object
dtypes: object(2)
memory usage: 15.6+ KB


In [32]:
eval_df.head()

,english,non_english
0,"Thank you so much, Chris.","Дуже дякую, Кріс!"
1,And it's truly a great honor to have the oppor...,"Справді, для мене це велика честь мати можливі..."
2,"I have been blown away by this conference, and...","Я в захваті від цієї конференції, і я хочу под..."
3,"And I say that sincerely, partly because (Mock...","І, щиро кажучи, частково тому що – (Схлипує) –..."
4,(Laughter) Put yourselves in my position.,(Сміх) Поставте себе на моє місце!
